# Calibrate Gages

Loop over all gages, calibrate, and export error stats. 

by Mike Durand, 5 March 2025


Dependencies:
* calval reaches dataframe. this notebook assumes that's stored in a calval-data directory
* experiment log dataframe, in this directory
* FLaPE-byrd repo pulled
* three .py files in a "src" directory, located in this directory
* various standard Python libraries
* NWIS tool installed for pulling USGS data (see notes in src)

This notebook supports option to either pull all data again (\~10 seconds total runtime per reach) or to analyze data already downloaded (\~ 0.1 seconds runtime/reach). It creates and assumes a directory structure, linked to the experiment log dataframe.

## 0 Imports & Functions

### 0.1 Imports

In [1]:
import os,sys
from pathlib import Path
import json
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import time

In [2]:
# FLaPE-Byrd imports
sys.path.append('/Users/mtd/GitHub/FLaPE-Byrd') 
from SWOTData import SWOTData,PullSWOT,ReadSWOT
from PullGageData import PullUSGS
from SWOTCalibrationUtilities import   OutputReachCalibration,CalibrateReach,SetOptions

In [3]:
# this automatically reloads things so you don't have to restart kernel after changing an import
%load_ext autoreload
%autoreload 2

## 1 Set up data and experiment log

### 1.0 Set up directories

In [7]:
CalValDir=Path('calval-data')
GageDir=Path('gage-data')
ExpDataDir=Path('ExperimentData')

### 1.1 Choose experiment and set switches

In [8]:
expdf=pd.read_csv(ExpDataDir.joinpath('experimentlog.csv'),index_col=0)
expdf

,expsid,expid,reachdomain,swotsource,slopedata,slopeminimum,slopeconsistencycheck,areaopt,constrainhw,flowlaw,darkfracmax
0,exps1,exp1-1,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000000,fd,False,MWAPN,0.4
1,exps1,exp1-2,connecticut+willamette calval gages,hydrochron,slope2,NaN,0.000000,fd,False,MWAPN,0.4
2,exps1,exp1-3,connecticut+willamette calval gages,hydrochron,slope,0.000017,-1.000000,fd,False,MWAPN,0.4
3,exps1,exp1-4,connecticut+willamette calval gages,hydrochron,slope,0.000034,-1.000000,fd,False,MWAPN,0.4
4,exps1,exp1-5,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000017,fd,False,MWAPN,0.4
5,exps1,exp1-6,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000000,fh,False,MWAPN,0.4
6,exps1,exp1-7,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000000,fh,True,MWAPN,0.4
7,exps1,exp1-8,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000000,fd,False,AHGD,0.4
8,exps1,exp1-9,connecticut+willamette calval gages,hydrochron,slope,NaN,0.000000,fd,False,MWAPN,0.2
9,exps2,exp2-1,usgs gages,hydrochron,slope,NaN,0.000000,fd,False,MWAPN,0.4


In [9]:
#choose experiment index to run
idx=0

In [10]:
# pull local switch settings out of dataframe. this could be improved...
expsid,expid,SWOTSource,fname_pvd,slope_data_element,slope_min,slope_const,area_option,constrainhw,\
    flowlaw,darkfracmax,reachdomain=SetOptions(expdf,idx)

running exp1-1
  domain: connecticut+willamette calval gages
  swot data source: hydrochron
  slope data element: slope
  minimum slope applied as filter: nan
  slope consistency check: 0.0
  area option: fd
  constrain height-width option: False
  flow law: MWAPN
  dark frac maximum: 0.4


### 1.2 Handle experiment folder

In [11]:
ExpsDir=ExpDataDir.joinpath(expsid)
ExpDir=ExpsDir.joinpath(expid)

In [12]:
# make new experiment set directory if it doesn't exist
if not os.path.isdir(ExpsDir):
    #make new directory
    os.makedirs(ExpsDir)

In [13]:
# make new directory if it doesn't exist
if not os.path.isdir(ExpDir):
    #make new directory
    os.makedirs(ExpDir)

In [14]:
dirname=ExpsDir.joinpath('unfiltered')
if not os.path.isdir(dirname):
    os.makedirs(dirname)    
else:
    print('warning! directory exists. check for old data before continuing')

warning! directory exists. check for old data before continuing


In [15]:
expdirs=['fit+stats','gage+SWOT','gage+SWOT+cal']

for expdir in expdirs:
    dirname=ExpDir.joinpath(expdir)
    if not os.path.isdir(dirname):
        os.makedirs(dirname)    
    else:
        print('warning! directory exists. check for old data before continuing. data will be overwritten')

warning! directory exists. check for old data before continuing. data will be overwritten
warning! directory exists. check for old data before continuing. data will be overwritten
warning! directory exists. check for old data before continuing. data will be overwritten


### 1.3 read in data

In [16]:
if reachdomain == 'connecticut+willamette calval gages':
    # read in list of calval reaches to run on
    fname=CalValDir.joinpath('calvalreaches.json')
elif reachdomain =='usgs gages':
    fname=GageDir.joinpath('gagereaches.json')

with open(fname) as file:
    reaches_to_run=json.load(file)

rids_to_run=list(reaches_to_run.keys())
rids_to_run.sort()
print('there are ', len(rids_to_run),'reaches to run')


there are  13 reaches to run


In [17]:
# read in SWORD
if SWOTSource=='ADT':
    swordfile='/Users/mtd/Data/SWOT/SWORD/SWORD_v17/Reaches_Nodes/netcdf/na_sword_v17.nc'
else:
    swordfile='/Users/mtd/Data/SWOT/SWORD/SWORD_v16/Reaches_Nodes/netcdf/na_sword_v16.nc'
sword_dataset = Dataset(swordfile)
swordreachids=sword_dataset["reaches/reach_id"][:]

In [18]:
# optionally read in SWORD translation file
if SWOTSource=='ADT':
    swordtranslation=pd.read_csv('/Users/mtd/Data/SWOT/SWORD/SWORD_v17/Reaches_Nodes/v16-17translation/NA_ReachIDs_v17_vs_v16.csv')
    swordtranslation.head()

### 1.5 Set up logging

In [19]:
logdest='file' # screen or file
if logdest=='file':
    logfname=ExpDir.joinpath('logfile.txt')
    logfile=open(logfname,'w')
elif logdest=='screen':
    logfile = sys.stdout # use this option to print logged output to screen

## 2 Loop over all reaches

SWOTSource:
* Hydrochron, then just pull data down from the web
* ADT, then pull from a dataframe with multiple reaches (need filename)
* offline, then pull from a directory with a dataframe for each reach (need directory)

In [20]:
# get data again by pulling from Hydrochron and USGS? Or just use data already pulled?
GetData=False #if false, then just use the "unfiltered" directory

In [21]:
# loop over reaches to run on and calibrate them

start_time = time.perf_counter()

print('starting calibration at',time.asctime(),file=logfile)

nmin=10
n_skip=0
n_run=0
outfreq=1 #prints screen status ever 'outfreq' files
for rid in rids_to_run:    
    # time and counting
    current_time = time.perf_counter()
    execution_time = current_time - start_time
    if (n_run+n_skip) % outfreq == 0:
        print('running ',n_run+n_skip,'/',len(rids_to_run),'execution time so far:',execution_time,'seconds')
    
    
    # just calibrate if a gage is assigned. else skip
    if not reaches_to_run[rid]['agency id']:
        n_skip+=1        
        continue
    usgsid=reaches_to_run[rid]['agency id']
    agency=reaches_to_run[rid]['agency']
    
    
    print('processing SWORD reach',rid,'which corresponds to ',agency,'id',usgsid,file=logfile )
    if SWOTSource=='ADT':
        rid17=int(swordtranslation[swordtranslation['v16_reach_id']==int(rid)]['v17_reach_id'])
        print('  mapping SWORD 16 reach ',rid,'to SWORD17 reach',rid17,file=logfile)
    
    
    fnameu=ExpsDir.joinpath('unfiltered').joinpath(rid+'_SWOT_gage_unfiltered.csv')
    
    if GetData:

        #2 pull USGS data
        QUSGS,nretrv=PullUSGS(usgsid,logfile)
        if nretrv==0:
            print('  no USGS data pulled. skipping',file=logfile)
            n_skip+=1
            continue        

        #3 get SWOT data
        if SWOTSource=='Hydrochron':
            SWOTdict=PullSWOT(rid,logfile)
        elif SWOTSource=='ADT':
            SWOTdict=ReadSWOT(rid,logfile,fname=fname_pvd,reachid17=rid17)

        SWOT=SWOTData(logfile,SWOTdict=SWOTdict)
        SWOT.MatchUSGS(QUSGS)
        # dump dataframe to "unfiltered"        
        SWOT.data['df'].to_csv(fnameu)        
        
    else:
        if not fnameu.exists():
            print('  file not found, skipping', file=logfile )
            n_skip+=1
            continue                    
        # 1-4: read in existing SWOT+gage data from "unfiltered" directory
        SWOT=SWOTData(logfile,SWOTfname=fnameu)
        if SWOTSource=='ADT':
            SWOT.data['reachid17']=rid17

    
    # 4 apply filters  
    if SWOTSource=='ADT':
        idx=list(swordreachids).index(np.int64(SWOT.data['reachid17']))
    else:
        idx=list(swordreachids).index(np.int64(rid))
        
    p_slope=sword_dataset['reaches']['slope'][idx] / 1000 # m/km -> m/m
    print('  SWORD slope=',p_slope*1e5,'cm/km',file=logfile )
    
    SWOT.threshold_filter(slope_min=slope_min,dark_max=darkfracmax)
    SWOT.bitwise_filter()
    SWOT.outlier_test()
    if slope_const >= 0:
        SWOT.slope_limit(p_slope,opt=slope_const)    
    
    # 5 filter data and save to a separate filtered dataframe to be read in by FLaPE-Byrd
    SWOT.data['df']=SWOT.data['df'][~SWOT.data['df']['bad']]
    SWOT.data['df']=SWOT.data['df'][SWOT.data['df']['Qgage'].notna()]
    
    fnameSG=ExpDir.joinpath('gage+SWOT').joinpath(rid+'_SWOT_gage.csv')
    SWOT.data['df'].to_csv(fnameSG)
    
    if len(SWOT.data['df'])<nmin:
        print('  no data remain after filtering. quitting',file=logfile )
        n_skip+=1
        continue    
    
    # 6 calibrate reach and save dataframe
    Cal = CalibrateReach(fnameSG,logfile,slope_opt=slope_data_element,
                         area_opt=area_option,constrainhw=constrainhw,
                         flowlawname=flowlaw,Verbose=False)
    SWOT.data['df']['Qhat']=list(Cal.Qhat)    
    # SWOT.data['df'].to_csv('cal_fits/gage+SWOT+cal/'+rid+'_SWOT_gage_cal.csv')    
    fnameSGc=ExpDir.joinpath('gage+SWOT+cal').joinpath(rid+'_SWOT_gage_cal.csv')
    SWOT.data['df'].to_csv(fnameSGc)
    
    # 7 save calibration parameters and error stats
    dfout=OutputReachCalibration(Cal,rid)
    # dfout.to_csv('cal_fits/fit+stats/'+rid+'_fits_stats.csv')    
    fnameout=ExpDir.joinpath('fit+stats').joinpath(rid+'_fits_stats.csv')
    dfout.to_csv(fnameout)
        
    # increment reach counter on success    
    print('  calibration complete.',file=logfile )
    n_run+=1

# close logfile and SWORD
print('Calibrations complete: ',n_run,'calibrations run, and ',n_skip,'skipped')
logfile.close()
sword_dataset.close()

running  0 / 13 execution time so far: 0.00048718899999933285 seconds
running  1 / 13 execution time so far: 0.0006129889999995086 seconds
running  2 / 13 execution time so far: 0.0006286499999994533 seconds
running  3 / 13 execution time so far: 0.0006430959999992325 seconds
running  4 / 13 execution time so far: 0.0777064429999994 seconds
running  5 / 13 execution time so far: 0.15185260099999986 seconds
running  6 / 13 execution time so far: 0.2244347839999996 seconds
running  7 / 13 execution time so far: 0.29678441099999997 seconds
running  8 / 13 execution time so far: 0.36791429799999964 seconds
running  9 / 13 execution time so far: 0.3679390229999999 seconds
running  10 / 13 execution time so far: 0.4456098989999999 seconds
running  11 / 13 execution time so far: 0.5218841049999998 seconds
running  12 / 13 execution time so far: 0.5986784799999993 seconds
Calibrations complete:  9 calibrations run, and  4 skipped
